In [1]:
import os
import json
from openai import AzureOpenAI
from datetime import datetime
from zoneinfo import ZoneInfo

#load the .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path='aoai.env')

# Debugging: Print environment variables to ensure they are loaded correctly
azure_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

print("AZURE_OPENAI_API_ENDPOINT:", azure_endpoint)
print("AZURE_OPENAI_API_KEY:", api_key)
print("AZURE_OPENAI_API_VERSION", api_version)

# Initialize the AzureOpenAI client
client = AzureOpenAI(
    azure_endpoint = azure_endpoint,
    api_key = api_key,
    api_version = api_version
)

# Define the deployment you want to use for chat completions
deployment_name = "gpt-4o"

# Do a simple prompt to the client to ensure it is working
'''
response = client.chat.completions.create(
    model=deployment_name,
    messages=[{"role": "user", "content": "Once upon a time"}]
)
'''

# print ("Response is:", response.choices[0].message.content)


AZURE_OPENAI_API_ENDPOINT: None
AZURE_OPENAI_API_KEY: None
AZURE_OPENAI_API_VERSION None


OpenAIError: Missing credentials. Please pass one of `api_key`, `azure_ad_token`, `azure_ad_token_provider`, or the `AZURE_OPENAI_API_KEY` or `AZURE_OPENAI_AD_TOKEN` environment variables.

# Get Current Time

In [44]:
# Simplified timezone data
TIMEZONE_DATA = {
    "tokyo": "Asia/Tokyo",
    "san francisco": "America/Los_Angeles",
    "paris": "Europe/Paris"
}

# Define the function to get the current time in a given timezone
def get_current_time(location):
    """Get the current time for a given location"""
    print(f"get_current_time called with location: {location}")  
    location_lower = location.lower()
    
    for key, timezone in TIMEZONE_DATA.items():
        if key in location_lower:
            print(f"Timezone found for {key}")  
            current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
            return json.dumps({
                "location": location,
                "current_time": current_time
            })
    
    print(f"No timezone data found for {location_lower}")  
    return json.dumps({"location": location, "current_time": "unknown"})

# Run Conversation with the user to get the current time in the format of HH:MM:SS

In [51]:
def run_conversation():
    print("Running conversation...")
    # Initial user message
    messages = [{"role": "user", "content": "What's the current time in San Francisco"}] # Single function call
    
    #messages = [{"role": "user", "content": "What's the current time in San Francisco, Tokyo, and Paris?"}] # Parallel function call with a single tool/function defined

    # Define the function for the model
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco",
                        },
                    },
                    "required": ["location"],
                },
            }
        }
    ]

    # First API call: Ask the model to use the function
    # print("Sending the first API call...")
    # print("Client is:", client)
    print("Messages are:", messages)
    # print("Deployment name is:", deployment_name)
    # print("Tools are:", tools)
    
    response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    # Process the model's response
    
    response_message = response.choices[0].message
    messages.append(response_message)

    print("Model's response.....")
    print(response_message)

    # Handle function calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            if tool_call.function.name == "get_current_time":
                function_args = json.loads(tool_call.function.arguments)
                print(f"Function arguments: {function_args}")  
                time_response = get_current_time(
                    location=function_args.get("location")
                )
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "get_current_time",
                    "content": time_response,
                })
    else:
        print("No tool calls were made by the model.")  

    # Second API call: Get the final response from the model
    final_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
    )

    return final_response.choices[0].message.content

# Run the conversation and print the result
print ("Entering run_conversation")
print(run_conversation())
print ("Exiting run_conversation")

Entering run_conversation
Running conversation...
Messages are: [{'role': 'user', 'content': "What's the current time in San Francisco"}]
Model's response.....
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jsc2KpaViuL7MoOrsyCN76QQ', function=Function(arguments='{"location":"San Francisco"}', name='get_current_time'), type='function')])
Function arguments: {'location': 'San Francisco'}
get_current_time called with location: San Francisco
Timezone found for san francisco
The current time in San Francisco is 3:20 AM.
Exiting run_conversation
